<a href="https://colab.research.google.com/github/johnnycleiton07/llm-studies/blob/main/Fine_tunning_na_pratica_com_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fine-tunning (ajuste fino) na prática com GPT

Fine-tuning em LLMs (Modelos de Linguagem Grande) envolve ajustar um modelo pré-treinado em um novo conjunto de dados específico para uma tarefa particular, refinando assim suas capacidades para melhor desempenho em contextos específicos sem necessidade de treinar o modelo do zero.

##Configurações iniciais

In [1]:
#gpu
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel , Trainer,  TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader
from datasets import load_dataset
import pandas as pd

* O código abaixo carrega e configura o modelo GPT-2 com seu tokenizador, define o token de padding e carrega um conjunto de dados do Wikitext-2, selecionando uma pequena amostra embaralhada de 1000 exemplos para treinamento.

In [3]:
model_name = "gpt2"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_name)

dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

small_dataset = dataset.shuffle(seed=42).select(range(1000))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

* Abaixo, converte a amostra de dados do conjunto Wikitext-2 em um DataFrame do Pandas e exibe as primeiras cinco linhas.

In [4]:
df = pd.DataFrame(small_dataset)
print(df.head())

                                                text
0   Continuous , short @-@ arc , high pressure xe...
1   Field Marshal Antonio José de Sucre is portra...
2   Norman Gary Finkelstein ( born December 8 , 1...
3   Galveston has several state @-@ funded charte...
4   Walpole 's works have not been completely neg...


* Este trecho abaixo define uma função para tokenizar os textos do conjunto de dados com padding e truncamento, e aplica essa função ao conjunto de dados amostrado, criando um novo conjunto de dados tokenizado.

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = small_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
#converte o conjunto de dados tokenizado em um DataFrame do Pandas e exibe as primeiras cinco linhas.
df = pd.DataFrame(tokenized_dataset)
print(df.head())

                                           input_ids  \
0  [45012, 837, 1790, 2488, 12, 31, 10389, 837, 1...   
1  [7663, 37899, 11366, 36997, 390, 47352, 260, 3...   
2  [19636, 10936, 4463, 7750, 5714, 357, 4642, 34...   
3  [5027, 4223, 261, 468, 1811, 1181, 2488, 12, 3...   
4  [6445, 36869, 705, 82, 2499, 423, 407, 587, 31...   

                                      attention_mask  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  


In [8]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

* Abaixo é criado um DataLoader para o conjunto de dados tokenizado, configurando um tamanho de lote de 4 e usando o data collator definido; em seguida, itera sobre o DataLoader e imprime as chaves e valores do primeiro lote.

In [9]:
batch_size = 4
data_loader = DataLoader(tokenized_dataset, batch_size=batch_size, collate_fn=data_collator)

for batch in data_loader:
    break

for key, value in batch.items():
    print(f"{key}: {value}")

input_ids: tensor([[45012,   837,  1790,  2488,    12,    31, 10389,   837,  1029,  3833,
         27132,   261, 10389, 32209,   423,   257,  3124,  5951,  7173,  5561,
         39204, 19613, 19606,   290,   389,   973,   287,  6591,   985, 24325,
           764,  1320,   318,   837,   262, 15358,  1512,   414,   286,   777,
         32209,  7173,  5561, 26748,   257, 16968,  2042,  1767, 50036,   326,
           468,   257,  5951,  1969,   284,   326,  6515,   422,   262,  3825,
           764,  2293,   484,   547,   717,  5495,  1141,   262, 16236,    82,
           837,   777, 32209,  2540, 13586,   262, 12238,  2488,    12,    31,
          5615,  6588, 10389, 32209,   287,  3807,  1628,   669,   764,  1119,
           389,  9322,   287,  7226,  3439,  3020,   837,   314, 22921,   290,
           262,   649,  4875,  1628,   669,  2646, 20128,  3341,   837, 29393,
           367,  2389, 46783,   837,  1029,  2488,    12,    31,   886,   366,
         16106,   366,  7644,  8091,   29

* Este código configura os argumentos de treinamento para um modelo, especificando o diretório de saída, número de épocas, tamanho do lote, e frequências de salvamento e logging.

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=50,
)

* Este código cria um `Trainer` para treinar o modelo usando os argumentos de treinamento especificados, o conjunto de dados tokenizado e o data collator.

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [12]:
trainer.train()

Step,Training Loss
50,4.075300
100,3.828800
150,3.772800
200,3.685200
250,3.623200
300,3.195300
350,3.103100
400,3.287100
450,3.148600
500,3.128200


TrainOutput(global_step=1250, training_loss=2.963734716796875, metrics={'train_runtime': 232.9876, 'train_samples_per_second': 21.46, 'train_steps_per_second': 5.365, 'total_flos': 326615040000000.0, 'train_loss': 2.963734716796875, 'epoch': 5.0})

* salva o modelo e o tokenizador ajustados no diretório `./fine_tuned_gpt2.`

In [13]:
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json')

* Este código carrega o modelo e o tokenizador ajustados a partir do diretório `./fine_tuned_gpt2`.

In [14]:
model_name = "./fine_tuned_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

* Esta função gera texto a partir de um prompt usando o modelo e tokenizador fornecidos, limitando o comprimento da saída e retornando o texto gerado como uma string.

In [15]:
def generate_text(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

* Este código usa a função `generate_text` para gerar um texto com base no prompt "Once upon a time" e imprime o texto gerado.

In [16]:
prompt = "Once upon a time"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Once upon a time, the British Empire was the most powerful nation in the world. The British Empire was the most powerful nation in the world, and the British Empire was the most powerful nation in the world. 
 
 
  
